In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install mysql-connector-python

This script is based on the YouTube crash course by Traversy Media [here](https://www.youtube.com/watch?v=BHwgnGEhYG8). Results of post each operation can be cross-checked in the MySQL Workbench GUI.

In [11]:
# database cell

import mysql.connector

config = {
    'user': 'root',
    'password': 'root@123',
    'host': 'localhost'
}

db = mysql.connector.connect(**config)
cursor = db.cursor()

In [12]:
# setup cell

from mysql.connector import errorcode

DB_NAME = 'acme'

TABLES = {}

TABLES['logs'] = (
    "CREATE TABLE `logs` ("
    "`id` int(11) NOT NULL AUTO_INCREMENT,"
    "`text` varchar(250) NOT NULL,"
    "`user` varchar(250) NOT NULL,"
    "`created` datetime NOT NULL DEFAULT CURRENT_TIMESTAMP,"
    "PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB"
)

def create_database():
    cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    print("Database {} created".format(DB_NAME))
    
def create_tables():
    cursor.execute("USE {}".format(DB_NAME))
    for table_name in TABLES:
        table_description = TABLES[table_name]
        try:
            print("Creating table ({}) ".format(table_name),end="")
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("Already Exists")
            else:
                print(err.msg)
                
                
create_database()
create_tables()

Database acme created
Creating table (logs) 

In [14]:
# reset cursor with database name

config = {
    'user': 'root',
    'password': 'root@123',
    'host': 'localhost',
    'database':'acme'
}

db = mysql.connector.connect(**config)
cursor = db.cursor(buffered=True)

In [15]:
# main functions to do Create Read Update Delete (CRUD) operations

In [16]:
# Create
def add_log(text,user):
    sql = ("INSERT INTO logs(text, user) VALUES (%s,%s)")
    cursor.execute(sql,(text,user,))
    db.commit()
    log_id = cursor.lastrowid
    print("Added log {}".format(log_id))
    
add_log("This is log one","Andy")
add_log("This is log two","Brad")
add_log("This is log three","Cris")
add_log("This is log four","Dave")
add_log("This is log five","Eva")

Added log 1
Added log 2
Added log 3
Added log 4
Added log 5


In [6]:
# Read

## Get all logs
def get_logs():
    sql = ("SELECT * FROM logs ORDER BY created DESC")
    cursor.execute(sql)
    result = cursor.fetchall()
    
    for row in result:
        print(row[1])

## Get only one log
def get_log(id):
    sql = ("SELECT * FROM logs WHERE id= %s")
    cursor.execute(sql,(id,))
    result = cursor.fetchone()
    
    for row in result:
        print(row)
        
get_logs()
get_log(2)

This is log one
This is log two
This is log three
This is log four
This is log five
2
This is log two
Brad
2021-08-19 13:18:27


In [17]:
# Update

def update_log(id,text):
    sql = ("UPDATE logs SET text = %s WHERE id = %s")
    cursor.execute(sql,(text,id,))
    db.commit()
    print("Log updated")

print("Pre-update: ")
get_logs()
update_log(2,'Updated log')
get_logs()

Pre-update: 
This is log one
This is log two
This is log three
This is log four
This is log five
Log updated
This is log one
Updated log
This is log three
This is log four
This is log five


In [18]:
# Delete

def delete_log(id):
    sql = ("DELETE FROM logs WHERE id = %s")
    cursor.execute(sql,(id,))
    db.commit()
    print("Log removed")
    
print("Pre-delete: ")
get_logs()
delete_log(2)
get_logs()

Pre-delete: 
This is log one
Updated log
This is log three
This is log four
This is log five
Log removed
This is log one
This is log three
This is log four
This is log five
